In [1]:
import pandas as pd
import numpy as np

In [2]:
import re

In [3]:
from pandas.testing import assert_series_equal

In [4]:
acct = pd.read_csv('FEEDVISOR_pre_daily_collection_snapshot_cleaned_pai_snapshot_c1_d2019-01-01_account.csv', dtype=object)
trgt = pd.read_csv('FEEDVISOR_pre_daily_collection_snapshot_cleaned_pai_snapshot_c1_d2019-01-01_target.csv', dtype=object)

In [5]:
big_acct = pd.read_csv('FEEDVISOR_new_pre_stitch_2019-01-09_pre_stitch_cleaned_account.csv', dtype=object)
big_trgt = pd.read_csv('FEEDVISOR_new_pre_stitch_2019-01-09_pre_stitch_cleaned_target_with_profiling.csv', dtype=object)

In [9]:
# assert (big_acct.index == big_trgt.index).all()
assert (acct.index == trgt.index).all()

In [13]:
big_acct.dropna(subset=['date'], inplace=True)
big_trgt.dropna(subset=['date'], inplace=True)
acct.dropna(subset=['date'], inplace=True)
trgt.dropna(subset=['date'], inplace=True)

big_acct.drop(big_acct[big_acct.date == '#NAME?'].index, inplace=True)
big_trgt.drop(big_trgt[big_trgt.date == '#NAME?'].index, inplace=True)

acct.next_calculated_exit_point = acct.next_calculated_exit_point.apply(lambda x: re.sub('-0', '-', x))

big_acct.churn_probability_date = big_acct.churn_probability_date.apply(
    lambda x: re.sub('^0', '', x) if pd.notnull(x) else x)

acct.churn_probability_date = pd.to_datetime(acct.churn_probability_date, format='%Y-%m-%d').astype(str)
big_acct.next_calculated_exit_point = pd.to_datetime(big_acct.next_calculated_exit_point, format='%Y-%m-%d').astype(str)
acct.next_calculated_exit_point = pd.to_datetime(acct.next_calculated_exit_point, format='%Y-%m-%d').astype(str)

In [8]:
for df in [big_acct, big_trgt, acct, trgt]:
    print(df.shape)

(1647607, 32)
(1651085, 16)
(68767, 26)
(68767, 17)


In [14]:
for df in [big_acct, big_trgt, acct, trgt]:
    df.date = pd.to_datetime(df.date, errors='coerce', format='%Y-%m-%d')

In [14]:
big_acct.date = big_acct.date.str.lstrip('SDR,')
big_acct.date = big_acct.date.str.rstrip('",')
big_trgt.date = big_trgt.date.str.lstrip('SDR,')
big_trgt.date = big_trgt.date.str.rstrip('",')

In [15]:
big_acct.cancellation_reason = big_acct.cancellation_reason.str.lstrip('"')
big_acct.cancellation_reason = big_acct.cancellation_reason.str.rstrip('"')

In [18]:
big_acct.churn_probability_date = pd.to_datetime(big_acct.churn_probability_date, format='%Y-%m-%d').astype(str)
acct.date = pd.to_datetime(acct.date, format='%Y-%m-%d').astype(str)
trgt.date = pd.to_datetime(trgt.date, format='%Y-%m-%d').astype(str)
big_acct.date = pd.to_datetime(big_acct.date, format='%Y-%m-%d').astype(str)
big_trgt.date = pd.to_datetime(big_trgt.date, format='%Y-%m-%d').astype(str)

In [19]:
# preprocessing!
trgt.churn_in_stage = trgt.churn_in_stage.astype(float)
trgt.churn_in_stage_paying = trgt.churn_in_stage_paying.astype(int)
trgt.churn_in_stage_poc = trgt.churn_in_stage_poc.astype(float)
trgt.reached_poc_stage = trgt.reached_poc_stage.astype(int)
trgt.repricing_activated = trgt.repricing_activated.astype(int)
trgt.pai_is_deleted = trgt.pai_is_deleted.astype(int)
trgt.account_birth_date = pd.to_datetime(trgt.account_birth_date, format='%Y-%m-%d').astype(str)
big_trgt.account_birth_date = pd.to_datetime(big_trgt.account_birth_date, format='%Y-%m-%d').astype(str)
trgt.churn_month = pd.to_datetime(trgt.churn_month, format='%Y-%m-%d').astype(str)
big_trgt.churn_month = pd.to_datetime(big_trgt.churn_month, format='%Y-%m-%d').astype(str)
trgt.last_activity = pd.to_datetime(trgt.last_activity, format='%Y-%m-%d').astype(str)
big_trgt.last_activity = pd.to_datetime(big_trgt.last_activity, format='%Y-%m-%d').astype(str)
trgt.poc_end_date = pd.to_datetime(trgt.poc_end_date, format='%Y-%m-%d').astype(str)
big_trgt.poc_end_date = pd.to_datetime(big_trgt.poc_end_date, format='%Y-%m-%d').astype(str)

In [20]:
for col in ['account_currency_code', 'base_mrr', 'last_bill_total_fee', 'last_bill_variable_fee', 
 'of_repricing_online_stores']:
    acct[col] = acct[col].astype(float)

In [21]:
acct.contract_term = acct.contract_term.apply(lambda x: str(float(x)) if not pd.isnull(float(x)) else x)

In [22]:
big_acct.contract_term = big_acct.contract_term.astype(str)

In [23]:
acct.contract_term = acct.contract_term.apply(lambda x: str(int(float(x))) if not pd.isnull(float(x)) else x)

In [24]:
big_acct.contract_term = big_acct.contract_term.astype(str)

In [25]:
acct.country = acct.country.map({'"Korea, Republic of"':'Korea, Republic of',
                  """\"Korea, Democratic People's Republic of\"""": "Korea, Democratic People's Republic of"})

In [57]:
# testing the groupby
t = pd.DataFrame(trgt.groupby('date')['last_activity'].apply(lambda x: x.mode())).reset_index()
t[t.level_1 != 0]

In [29]:
acct.set_index(['pai_account_id', 'date'], inplace=True)
trgt.set_index(['pai_account_id', 'date'], inplace=True)
big_acct.set_index(['pai_account_id', 'date'], inplace=True)
big_trgt.set_index(['pai_account_id', 'date'], inplace=True)

Reset index if needed

In [27]:
acct.reset_index(inplace=True)
big_acct.reset_index(inplace=True)
trgt.reset_index(inplace=True)
big_trgt.reset_index(inplace=True)

In [28]:
acct['id'] = acct['pai_account_id'] + acct['date'].astype(str)
trgt['id'] = trgt['pai_account_id'] + trgt['date'].astype(str)

big_acct['id'] = big_acct['pai_account_id'] + big_acct['date'].astype(str)
big_trgt['id'] = big_trgt['pai_account_id'] + big_trgt['date'].astype(str)

In [30]:
set(big_acct.id.values) - set(big_trgt.id.values)

set()

In [31]:
set(acct.id.values) - set(trgt.id.values)

set()

In [32]:
set(big_trgt.id.values) - set(big_acct.id.values)

{nan,
 '001b000003MdPRR2017-03-19',
 '001b000000h47Yl2017-04-16',
 '001b000000IxxsE2017-03-19',
 '001b000000c5Oef2017-04-25',
 '001b0000008ca9l2017-04-25',
 '001b000003Zr5B62017-03-19',
 '001b000003EgQai2017-03-19',
 '001b0000005cS8o2017-04-16',
 '001b000000Vr8wh2017-04-16',
 '001b000000ygXM22017-04-25',
 '001b00000054ILE2017-04-25',
 '001b000001SGqhW2017-04-25',
 '001b000003Kjbz02017-04-16',
 '001b000001pm0dJ2017-04-25',
 '001b0000008cHQV2017-04-16',
 '001b0000001y4872017-03-19',
 '001b000000dEng92017-03-19',
 '001b0000015m5BU2017-04-25',
 '001b000000fUfmQ2017-04-16',
 '001b000003j4pGR2017-04-25',
 '001b000000S6fj32017-03-19',
 '001b000003TML3M2017-03-19',
 '001b000000c5GPB2017-04-16',
 '001b000001EhVno2017-04-25',
 '001b000000khVDk2017-03-19',
 '001b000000I5YL02017-03-19',
 '001b000000bPeUH2017-04-25',
 '001b000001MVJ452017-04-25',
 '001b00000054UNa2017-04-25',
 '001b0000019v5xx2017-03-19',
 '001b000001SFnGP2017-04-16',
 '001b000000sCHBK2017-03-19',
 '001b000000LTh2c2017-04-16',
 '00

In [33]:
set(trgt.id.values) - set(acct.id.values)

set()

In [253]:
len(set(trgt.id.values) - set(big_trgt.id.values))

18252

In [25]:
new_less_old = set(big_trgt.loc[big_trgt.pai_is_deleted == '0'].id.values) - set(trgt.id.values)

In [26]:
len(new_less_old)

75838

In [38]:
t = trgt.loc[trgt.index.difference(big_trgt.index)].reset_index()
t.date = pd.to_datetime(t.date)
t = t.set_index('date')
t.loc[:'2018-02-09']['pai_account_id'].nunique()

In [27]:
new_old_intersection = set(big_trgt.id.values) & set(trgt.id.values)

In [28]:
len(new_old_intersection)

50507

In [29]:
len(new_less_old) + len(new_old_intersection)

126345

In [30]:
old_less_new = set(trgt.id.values) - set(big_trgt.id.values)

In [31]:
len(old_less_new)

18260

In [32]:
x = trgt[trgt.id.isin(list(old_less_new))].reset_index().groupby('date').count()

In [33]:
x[x.pai_account_id > 1].sort_index()

,pai_account_id,client_id,recipe_execution_date,last_activity,churn_month,repricing_activated,profile,poc_end_date,churn_in_stage_poc,reached_poc_stage,stage,churn_in_stage_paying,churn_in_stage,account_birth_date,is_deleted,pai_is_deleted,id
date,,,,,,,,,,,,,,,,,
2017-03-19,519,519,519,519,272,519,517,519,407,519,519,519,516,519,519,519,519
2017-04-16,549,549,549,548,284,549,548,549,436,549,549,549,548,549,549,549,549
2017-04-25,556,556,556,555,288,556,555,556,443,556,556,556,554,556,556,556,556
2018-02-10,689,689,689,688,386,689,687,689,577,689,689,689,688,689,689,689,689
2018-02-11,689,689,689,688,386,689,687,689,577,689,689,689,688,689,689,689,689
2018-02-12,689,689,689,688,386,689,687,689,577,689,689,689,688,689,689,689,689
2018-02-13,689,689,689,688,386,689,687,689,577,689,689,689,688,689,689,689,689
2018-02-14,689,689,689,688,386,689,687,689,577,689,689,689,688,689,689,689,689
2018-02-17,689,689,689,688,388,689,687,689,577,689,689,689,688,689,689,689,689


In [87]:
dates_to_check = x[x.pai_account_id == 1].index.tolist()

In [96]:
trgt.xs(dates_to_check[0], level=1).head()

,client_id,recipe_execution_date,last_activity,churn_month,repricing_activated,profile,poc_end_date,churn_in_stage_poc,reached_poc_stage,stage,churn_in_stage_paying,churn_in_stage,account_birth_date,is_deleted,pai_is_deleted,id
pai_account_id,,,,,,,,,,,,,,,,
001b000003EgQai,1,2018-03-18,2017-10-2,NaN,1,smb,2016-5-1,0.0,1,paying,0,0.0,2016-1-31,false,0,001b000003EgQai2017-10-04
001b000001Eh9qL,1,2018-03-18,2016-8-18,2016-8-1,0,smb,2015-12-17,NaN,1,paying,1,1.0,2015-9-17,false,0,001b000001Eh9qL2017-10-04
001b000003EgXAM,1,2018-03-18,2016-8-10,2016-8-1,0,smb,2016-7-6,1.0,1,poc,0,1.0,2016-4-6,false,0,001b000003EgXAM2017-10-04
001b000001EhBMH,1,2018-03-18,2017-9-12,NaN,1,enterprise,2016-3-30,0.0,0,paying,0,0.0,2016-3-29,false,0,001b000001EhBMH2017-10-04
001b000003ODOhK,1,2018-03-18,2017-8-21,2016-9-1,0,smb,2016-8-30,1.0,1,poc,0,1.0,2016-5-30,false,0,001b000003ODOhK2017-10-04


In [104]:
issues = {}
for date in dates_to_check:
    old = set(trgt.xs(dates_to_check[0], level=1).index)
    new = set(big_trgt.xs(dates_to_check[0], level=1).index)
    issues[date] = old - new

In [94]:
trgt.loc(axis=0)[:,dates_to_check[0]]

,,client_id,recipe_execution_date,last_activity,churn_month,repricing_activated,profile,poc_end_date,churn_in_stage_poc,reached_poc_stage,stage,churn_in_stage_paying,churn_in_stage,account_birth_date,is_deleted,pai_is_deleted,id
pai_account_id,date,,,,,,,,,,,,,,,,
001b000003EgQai,2017-10-04,1,2018-03-18,2017-10-2,NaN,1,smb,2016-5-1,0.0,1,paying,0,0.0,2016-1-31,false,0,001b000003EgQai2017-10-04
001b000001Eh9qL,2017-10-04,1,2018-03-18,2016-8-18,2016-8-1,0,smb,2015-12-17,NaN,1,paying,1,1.0,2015-9-17,false,0,001b000001Eh9qL2017-10-04
001b000003EgXAM,2017-10-04,1,2018-03-18,2016-8-10,2016-8-1,0,smb,2016-7-6,1.0,1,poc,0,1.0,2016-4-6,false,0,001b000003EgXAM2017-10-04
001b000001EhBMH,2017-10-04,1,2018-03-18,2017-9-12,NaN,1,enterprise,2016-3-30,0.0,0,paying,0,0.0,2016-3-29,false,0,001b000001EhBMH2017-10-04
001b000003ODOhK,2017-10-04,1,2018-03-18,2017-8-21,2016-9-1,0,smb,2016-8-30,1.0,1,poc,0,1.0,2016-5-30,false,0,001b000003ODOhK2017-10-04
001b000003jw5h0,2017-10-04,1,2018-03-18,2017-7-30,2017-9-29,0,smb,2017-3-29,0.0,0,paying,1,1.0,2017-3-28,false,0,001b000003jw5h02017-10-04
001b000003EjIPW,2017-10-04,1,2018-03-18,2017-10-2,2016-5-1,0,smb,2016-4-15,1.0,1,poc,0,1.0,2016-1-15,false,0,001b000003EjIPW2017-10-04
001b000003GZ2GR,2017-10-04,1,2018-03-18,2017-9-20,NaN,1,smb,2016-4-30,0.0,1,paying,0,0.0,2016-1-18,false,0,001b000003GZ2GR2017-10-04
001b000003ODP7X,2017-10-04,1,2018-03-18,2017-9-26,2016-9-1,0,enterprise,2016-8-31,1.0,1,poc,0,1.0,2016-5-31,false,0,001b000003ODP7X2017-10-04


In [221]:
trgt[trgt.id.isin(list(old_less_new))]

,,client_id,recipe_execution_date,last_activity,churn_month,repricing_activated,profile,poc_end_date,churn_in_stage_poc,reached_poc_stage,stage,churn_in_stage_paying,churn_in_stage,account_birth_date,is_deleted,pai_is_deleted,id
pai_account_id,date,,,,,,,,,,,,,,,,
001b000003mZ0Qi,2018-1-29,1,2018-03-18,2018-1-28,NaN,1,enterprise,2017-8-25,0.0,1,paying,0,0.0,2017-5-11,false,0,001b000003mZ0Qi2018-1-29
0010J00001oc8VO,2018-1-29,1,2018-03-18,2018-1-28,NaN,1,smb,2018-1-1,0.0,1,paying,0,0.0,2017-9-28,false,0,0010J00001oc8VO2018-1-29
001b000000NuiqD,2018-1-29,1,2018-03-18,2018-1-24,NaN,1,smb,2017-12-1,0.0,1,paying,0,0.0,2017-8-1,false,0,001b000000NuiqD2018-1-29
001b000003nCs6A,2018-1-29,1,2018-03-18,2018-1-12,NaN,0,smb,2018-3-15,0.0,1,poc,0,0.0,2017-8-16,false,0,001b000003nCs6A2018-1-29
0010J00001ooUjf,2018-1-29,1,2018-03-18,2031-12-23,NaN,1,enterprise,2018-3-10,0.0,1,poc,0,0.0,2017-12-10,false,0,0010J00001ooUjf2018-1-29
0010J00001oqgkS,2018-1-29,1,2018-03-18,2018-3-14,NaN,1,enterprise,2018-3-22,0.0,1,poc,0,0.0,2017-12-22,false,0,0010J00001oqgkS2018-1-29
001b000003nCoYZ,2018-1-29,1,2018-03-18,2018-1-24,NaN,0,smb,2018-4-19,0.0,1,poc,0,0.0,2018-1-20,false,0,001b000003nCoYZ2018-1-29
0010J00001oq1Ks,2018-1-29,1,2018-03-18,2018-1-26,NaN,1,smb,2018-6-2,0.0,1,poc,0,0.0,2017-12-3,false,0,0010J00001oq1Ks2018-1-29
0010J00001oc8DF,2018-1-29,1,2018-03-18,2018-1-22,NaN,1,smb,2017-12-19,0.0,1,paying,0,0.0,2017-9-13,false,0,0010J00001oc8DF2018-1-29


In [56]:
t = pd.Series(sorted(trgt[trgt.id.isin(list(diff_by_id))].reset_index().date.unique()))